<a href="https://colab.research.google.com/github/jchen8000/DemystifyingLLMs/blob/main/6_Deployment/Chatbot_FLAN_T5_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6. Deployment of LLMs

## 5.11 Chatbot, Example of LLM-Powered Application

**Chatbot built with local LLM, flan-t5-base plus LoRA.**

Make sure flan-t5-base_lora model is in the output folder

In [ ]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers --quiet
!pip install peft --quiet
!pip install sentencepiece --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 13.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import GenerationConfig
from peft import PeftModel, PeftConfig

In [ ]:
# Load the FLAN-T5-Base model and tokenizer
base_model_name = 'google/flan-t5-base'
lora_model_name = '../5_Fine-Tuning/outputs/flan-t5-base_lora'

def load_model(base_model_name, lora_model_name):
    tokenizer = T5Tokenizer.from_pretrained(base_model_name)
    base_model = T5ForConditionalGeneration.from_pretrained(base_model_name,
                                                       torch_dtype=torch.bfloat16)
    model = PeftModel.from_pretrained(base_model,
                                      lora_model_name,
                                      torch_dtype=torch.bfloat16,
                                      is_trainnable=False)

    return tokenizer, model

def generate_output(tokenizer, model, input_text, max_length=200):
    # Tokenize the input text and generate the model's output
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(max_new_tokens=200, num_beams=1) )

    # Decode the generated tokens to a string
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

tokenizer, model = load_model(base_model_name, lora_model_name)


total_parameter = sum(p.numel() for p in model.parameters())
print(f"Parameters of the model: {total_parameter:,}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Parameters of the model: 249,347,328


In [ ]:
def chatbot():

    print("Chatbot initialized. You can start chatting now (type 'quit' to stop)!\n")

    while True:
        # Get user input
        user_input = input("You: ")

        # Check if the user wants to quit
        if user_input.lower() == "quit":
            break

        response = generate_output(tokenizer, model, user_input)

        # Print the generated text
        print(f"Chatbot: {response}\n")

# Run the chatbot
chatbot()

Chatbot initialized. You can start chatting now (type 'quit' to stop)!

You: How are you doing today?
Chatbot: You are doing well today.

You: Summarize:  saludos,es que tengo un problema con mi iPhone,y lo que ocurre es que no puedo llamar ni recibir llamadas,y no s qu ms hacer,ya que acud a Apple Support y aunque intenten llamarme,no entran llamadas,quiero saber cual es su recomendacin.Gracias. <BR> We offer support via Twitter in English. Get help in Spanish here <LINK> or join <LINK> <BR> AppleSupport I can speak English so so,can you help me?i have an iPhone 7,and 1 week ago,I cant make calls or receive them,what I do? <BR> Being able to make and receive calls is important and wed like to help. Which exact iOS version is installed on it and have you tried any steps yet? <BR> AppleSupport I have iOS 11.1.2 and I have tried everything what the page said.im Colombian. <BR> Is this the page youve tried all the steps from <LINK> yes, what did you find out when you contacted your carrie

Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors


Chatbot: Advice: I have a problem with my iPhone and I can't make calls or receive them,what I do?

You: quit
